In [1]:
import sys
print(sys.path)

from src.training import SegmentationTrainer, TrainingConfig
from src.models import build_dual_encoder_mobilenet_unet

['/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages']


In [2]:
config = TrainingConfig(
      data_root="datasets/NYUDepthv2",
      use_rgb=True,
      use_depth=True,
      lr=1e-3,
      epochs=20,
      batch_size=32,
      early_stopping_enabled=True,
      early_stopping_patience=5,
      device="auto",
      save_best_path="checkpoints/dual_encoder_unet_mobilenet/best.pth",
      save_latest_path="checkpoints/dual_encoder_unet_mobilenet/latest.pth",
      num_classes=40,
)

# Create model and train
model = build_dual_encoder_mobilenet_unet(num_classes=config.num_classes, variant='small')

trainer = SegmentationTrainer(config)

Loading ImageNet pretrained MobileNetV3-Small weights...
✓ Successfully loaded ImageNet pretrained MobileNetV3-Small weights
Loading ImageNet pretrained MobileNetV3-Small weights for depth encoder...
✓ Successfully loaded ImageNet pretrained MobileNetV3-Small weights
✓ Adapted first conv layer for depth input using weighted RGB initialization (green-favored)
Dual-Encoder UNet-MobileNetV3-SMALL initialized:
  Total parameters: 3,321,105
  Trainable parameters: 3,321,105
  RGB encoder channels: [16, 16, 24, 48, 96]
  Depth encoder channels: [16, 16, 24, 48, 96]
  Fused channels: [16, 16, 24, 48, 96]
Using device: mps
EarlyStopping: monitoring 'val_miou' with patience=5 (mode=max)


In [3]:
model.freeze_backbone()
trained_model = trainer.train(model)

MobileNetV3-small encoder frozen: parameters will not be updated during training
MobileNetV3-small depth encoder frozen: parameters will not be updated during training
Training configuration:
  Use RGB: True
  Use Depth: True
  Input channels: 4
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 3,321,105
Trainable parameters: 1,467,377
Starting training for 20 epochs (from epoch 1)...


/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoch 1 | Iter 00001] loss=3.7611
[Epoch 1 | Iter 00002] loss=3.7606
[Epoch 1 | Iter 00003] loss=3.7483
[Epoch 1 | Iter 00004] loss=3.7390
[Epoch 1 | Iter 00005] loss=3.7318
[Epoch 1 | Iter 00006] loss=3.7234
[Epoch 1 | Iter 00007] loss=3.7167
[Epoch 1 | Iter 00008] loss=3.7093
[Epoch 1 | Iter 00009] loss=3.6991
[Epoch 1 | Iter 00010] loss=3.6907
[Epoch 1 | Iter 00011] loss=3.6805
[Epoch 1 | Iter 00012] loss=3.6716
[Epoch 1 | Iter 00013] loss=3.6625
[Epoch 1 | Iter 00014] loss=3.6522
[Epoch 1 | Iter 00015] loss=3.6417
[Epoch 1 | Iter 00016] loss=3.6308
[Epoch 1 | Iter 00017] loss=3.6216
[Epoch 1 | Iter 00018] loss=3.6108
[Epoch 1 | Iter 00019] loss=3.6003
[Epoch 1 | Iter 00020] loss=3.5908
[Epoch 1 | Iter 00021] loss=3.5793
[Epoch 1 | Iter 00022] loss=3.5671
[Epoch 1 | Iter 00023] loss=3.5547
[Epoch 1 | Iter 00024] loss=3.5433
[Epoch 1 | Iter 00025] loss=3.5317
[Epoch 1] Train: loss=2.9916 mIoU=0.0355 pixacc=0.3624 (36.2s+27.5s, 22.0 smp/s) | Val: loss=2.8344 mIoU=0.0317 pixacc=0.3719

In [4]:
model.unfreeze_backbone()
config
config.epochs = 100
config.lr = 1e-4 # lower learning rate for fine-tuning
trainer = SegmentationTrainer(config)
trained_model = trainer.train(model)

MobileNetV3-small encoder unfrozen: parameters will be updated during training
MobileNetV3-small depth encoder unfrozen: parameters will be updated during training
Using device: mps
EarlyStopping: monitoring 'val_miou' with patience=5 (mode=max)
Training configuration:
  Use RGB: True
  Use Depth: True
  Input channels: 4
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 3,321,105
Trainable parameters: 3,321,105
Resuming training from: checkpoints/dual_encoder_unet_mobilenet/latest.pth
✓ Resumed from epoch 20
✓ Training history loaded with 20 epochs
✓ Best mIoU so far: 0.1089
🔧 Applied config overrides:
  learning_rate: 0.001000 → 0.000100
Starting training for 100 epochs (from epoch 21)...
[Epoch 21 | Iter 00001] loss=1.2825
[Epoch 21 | Iter 00002] loss=1.2903
[Epoch 21 | Iter 00003] loss=1.3282
[Epoch 21 | Iter 00004] loss=1.3259
[Epoch 21 | Iter 00005] loss=1.3229
[Epoch 21 | Iter 00006] loss=1.3274
[Epoch 21 | Iter 00007] loss=1.3

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/src/datasets/nyu_depth.py", line 95, in __getitem__
    sample = self.transform(sample)
  File "/Users/reubendrum

In [3]:
model.unfreeze_backbone()
config.epochs = 100
config.lr = 1e-4 # lower learning rate for fine-tuning
trainer = SegmentationTrainer(config)
trained_model = trainer.train(model)

MobileNetV3-small encoder unfrozen: parameters will be updated during training
MobileNetV3-small depth encoder unfrozen: parameters will be updated during training
Using device: mps
EarlyStopping: monitoring 'val_miou' with patience=5 (mode=max)
Training configuration:
  Use RGB: True
  Use Depth: True
  Input channels: 4
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 3,321,105
Trainable parameters: 3,321,105
Resuming training from: checkpoints/dual_encoder_unet_mobilenet/latest.pth
✓ Resumed from epoch 45
✓ Training history loaded with 45 epochs
✓ Best mIoU so far: 0.1359
✓ No optimizer parameter changes needed
Starting training for 100 epochs (from epoch 46)...


/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoch 46 | Iter 00001] loss=1.1985
[Epoch 46 | Iter 00002] loss=1.2136
[Epoch 46 | Iter 00003] loss=1.2265
[Epoch 46 | Iter 00004] loss=1.2176
[Epoch 46 | Iter 00005] loss=1.2083
[Epoch 46 | Iter 00006] loss=1.2112
[Epoch 46 | Iter 00007] loss=1.2165
[Epoch 46 | Iter 00008] loss=1.2360
[Epoch 46 | Iter 00009] loss=1.2357
[Epoch 46 | Iter 00010] loss=1.2374
[Epoch 46 | Iter 00011] loss=1.2276
[Epoch 46 | Iter 00012] loss=1.2227
[Epoch 46 | Iter 00013] loss=1.2247
[Epoch 46 | Iter 00014] loss=1.2258
[Epoch 46 | Iter 00015] loss=1.2149
[Epoch 46 | Iter 00016] loss=1.2179
[Epoch 46 | Iter 00017] loss=1.2190
[Epoch 46 | Iter 00018] loss=1.2135
[Epoch 46 | Iter 00019] loss=1.2198
[Epoch 46 | Iter 00020] loss=1.2190
[Epoch 46 | Iter 00021] loss=1.2193
[Epoch 46 | Iter 00022] loss=1.2137
[Epoch 46 | Iter 00023] loss=1.2094
[Epoch 46 | Iter 00024] loss=1.2112
[Epoch 46 | Iter 00025] loss=1.2050
[Epoch 46] Train: loss=1.1421 mIoU=0.2226 pixacc=0.6693 (48.8s+26.7s, 16.3 smp/s) | Val: loss=1.6238

In [ ]:
# reload SegmentationTrainer with new config
model.unfreeze_backbone()
config_stage_3 = config
config_stage_3.epochs = 100 # go essentially until early stopping
config_stage_3.lr = 1e-4 # lower learning rate for fine-tuning
trainer = SegmentationTrainer(config_stage_3)
trained_model = trainer.train(model)